In [1]:
%load_ext autoreload 
%autoreload 2 

import numpy as np 
import json
import re
import string 

import tensorflow as tf 
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

from tensorflow.keras import layers, models, callbacks, losses

2023-07-24 05:02:41.685004: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 05:02:41.772814: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-24 05:02:41.794236: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-24 05:02:42.122726: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

1 Physical GPUs, 1 Logical GPUs


2023-07-24 05:02:42.648196: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-24 05:02:42.668350: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-24 05:02:42.668462: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-24 05:02:42.668964: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

### PARAMETERS

In [2]:
VOCAB_SIZE = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100
N_UNITS = 128
VALIDATION_SPLIT = 0.2 
SEED = 42 
LOAD_MODEL = False 
BATCH_SIZE = 32 
EPOCHS = 25 


### Load the data 

In [3]:
with open("../data/epirecipes/full_format_recipes.json") as json_data:
    recipe_data = json.load(json_data)

In [4]:
# Filter the dataset
filtered_data = [
    "Recipe for " + x["title"] + " | " + " ".join(x["directions"])
    for x in recipe_data
    if "title" in x
    and x["title"] is not None
    and "directions" in x
    and x["directions"] is not None
]

In [5]:
filtered_data[0]

'Recipe for Lentil, Apple, and Turkey Wrap  | 1. Place the stock, lentils, celery, carrot, thyme, and salt in a medium saucepan and bring to a boil. Reduce heat to low and simmer until the lentils are tender, about 30 minutes, depending on the lentils. (If they begin to dry out, add water as needed.) Remove and discard the thyme. Drain and transfer the mixture to a bowl; let cool. 2. Fold in the tomato, apple, lemon juice, and olive oil. Season with the pepper. 3. To assemble a wrap, place 1 lavash sheet on a clean work surface. Spread some of the lentil mixture on the end nearest you, leaving a 1-inch border. Top with several slices of turkey, then some of the lettuce. Roll up the lavash, slice crosswise, and serve. If using tortillas, spread the lentils in the center, top with the turkey and lettuce, and fold up the bottom, left side, and right side before rolling away from you.'

In [6]:
#count the recipes 
n_recipes = len(filtered_data)
print(f"{n_recipes} recipes loaded")

20111 recipes loaded


### Tokenize the data 

Tokenization is the process of splitting the text to individual units (either words, or characters)

In [7]:
# Pad the punctuation to treat them as separate 'words' 

def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s

text_data = [pad_punctuation(x) for x in filtered_data]

In [8]:
text_data[0]

'Recipe for Lentil , Apple , and Turkey Wrap | 1 . Place the stock , lentils , celery , carrot , thyme , and salt in a medium saucepan and bring to a boil . Reduce heat to low and simmer until the lentils are tender , about 30 minutes , depending on the lentils . ( If they begin to dry out , add water as needed . ) Remove and discard the thyme . Drain and transfer the mixture to a bowl ; let cool . 2 . Fold in the tomato , apple , lemon juice , and olive oil . Season with the pepper . 3 . To assemble a wrap , place 1 lavash sheet on a clean work surface . Spread some of the lentil mixture on the end nearest you , leaving a 1 - inch border . Top with several slices of turkey , then some of the lettuce . Roll up the lavash , slice crosswise , and serve . If using tortillas , spread the lentils in the center , top with the turkey and lettuce , and fold up the bottom , left side , and right side before rolling away from you . '

In [9]:
# Convert to a Tensorflow Dataset

text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [10]:
#Create a vectorisation layer 
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens = VOCAB_SIZE,
    output_mode="int",
    output_sequence_length= MAX_LEN + 1 ,
)

In [11]:
#Adapt the layer to the training set 
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [12]:
#Display some token:word mapping 
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: .
3: ,
4: and
5: to
6: in
7: the
8: with
9: a


In [13]:
#display the same example converted to ints 
example_data = text_data[9]
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[  26   16  557    1    8  298  335  189    4 1054  494   27  332  228
  235  262    5  594   11  133   22  311    2  332   45  262    4  671
    4   70    8  171    4   81    6    9   65   80    3  121    3   59
   12    2  299    3   88  650   20   39    6    9   29   21    4   67
  529   11  164    2  320  171  102    9  374   13  643  306   25   21
    8  650    4   42    5  931    2   63    8   24    4   33    2  114
   21    6  178  181 1245    4   60    5  140  112    3   48    2  117
  557    8  285  235    4  200  292  980    2  107  650   28   72    4
  108   10  114    3   57  204   11  172    2   73  110  482    3  298
    3  190    3   11   23   32  142   24    3    4   11   23   32  142
   33    6    9   30   21    2   42    6  353    3 3224    3    4  150
    2  437  494    8 1281    3   37    3   11   23   15  142   33    3
    4   11   23   32  142   24    6    9  291  188    5    9  412  572
    2  230  494    3   46  335  189    3   20  557    2    0    0    0
    0 

### Creating the Training Set 


In [14]:
#Create the training sets and the same text shifted by one word 

def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:,:-1]
    y = tokenized_sentences[:, 1:]
    return x,y

train_ds = text_ds.map(prepare_inputs)

In [15]:
train_ds

<MapDataset element_spec=(TensorSpec(shape=(None, 200), dtype=tf.int64, name=None), TensorSpec(shape=(None, 200), dtype=tf.int64, name=None))>

### Build the LSTM 

The input is the sequence of integer tokens and the output is the probability of each word in the vocabulary appearing next in the sequence. 

An embedding layer is a lookup table that converts each integer token into a vector length of embedding size.

The input layer passes a tensor of integer sequences of shape [batch_size, seq_length] to the embedding layer and the output will be [batch_size,seq_length,embedding_size]. 

In [16]:
inputs = layers.Input(shape=(None, ), dtype="int32")
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x = layers.LSTM(N_UNITS, return_sequences=True)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         1000000   
                                                                 
 lstm (LSTM)                 (None, None, 128)         117248    
                                                                 
 dense (Dense)               (None, None, 10000)       1290000   
                                                                 
Total params: 2,407,248
Trainable params: 2,407,248
Non-trainable params: 0
_________________________________________________________________


In [17]:
if LOAD_MODEL:
    # model.load_weights('./models/model')
    lstm = models.load_model("./models/lstm", compile=False)

### Train the LSTM

In [18]:
loss_fn = losses.SparseCategoricalCrossentropy()
lstm.compile("adam", loss_fn)

In [19]:
#Create a TextGenerator Checkpoint 
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }  # <1>

    def sample_from(self, probs, temperature):  # <2>
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]  # <3>
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:  # <4>
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)  # <5>
            sample_token, probs = self.sample_from(y[0][-1], temperature)  # <6>
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)  # <7>
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("recipe for", max_tokens=100, temperature=1.0)

In [20]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.ckpt",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [21]:
lstm.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/25


2023-07-24 05:09:37.048835: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400
2023-07-24 05:09:37.169350: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


628/629 [============================>.] - ETA: 0s - loss: 4.4260
generated text:
recipe for tap wave tomato in cook aside , 10 chile , heat until held some tablespoons chopped garlic | and simmer and needed . then green over tomato about / into mixer . inch minutes at broilerproof sharp a towel is be stand vinegar over large adding salt separately . spread bowl 

629/629 [==============================] - 18s 25ms/step - loss: 4.4246
Epoch 2/25
628/629 [============================>.] - ETA: 0s - loss: 3.1509
generated text:
recipe for glaze sandwiches with orange matcha with beef and cucumber | combine sugar first until be serving pulp and kept let grilled the much sticky pan . freeze uncovered , stirring softened , stirring occasionally once in 3 minutes per retro , until listed , about 1 / 4 - inch skillet . heat niçoise in a high heat until simmer , 1 minute . drain mixture . drizzle until ingredient heated through and refrigerate until warm , turning , about 2 minutes . add turke

In [22]:
# Save the final model
lstm.save("./models/lstm")

INFO:tensorflow:Assets written to: ./models/lstm/assets


INFO:tensorflow:Assets written to: ./models/lstm/assets


In [23]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\nPROMPT: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [24]:
info = text_generator.generate(
    "recipe for roasted vegetables | chop 1 /", max_tokens=10, temperature=1.0
)


generated text:
recipe for roasted vegetables | chop 1 / 2 teaspoon



In [25]:
print_probs(info, vocab)


PROMPT: recipe for roasted vegetables | chop 1 /
2:   	47.4%
4:   	42.58%
3:   	5.39%
8:   	2.92%
2cup:   	0.5%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 2
cup:   	72.91%
teaspoon:   	8.81%
tsp:   	7.83%
tablespoon:   	3.27%
stick:   	1.15%
--------



In [26]:
info = text_generator.generate(
    "recipe for roasted vegetables | chop 1 /", max_tokens=10, temperature=0.2
)


generated text:
recipe for roasted vegetables | chop 1 / 2 cup



In [27]:
print_probs(info, vocab)


PROMPT: recipe for roasted vegetables | chop 1 /
2:   	63.09%
4:   	36.91%
3:   	0.0%
8:   	0.0%
2cup:   	0.0%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 2
cup:   	100.0%
teaspoon:   	0.0%
tsp:   	0.0%
tablespoon:   	0.0%
stick:   	0.0%
--------

